In [1]:
from utils.SystemMonitor import SystemMonitor
from utils.Table import Table

import os
import tensorflow as tf
from importlib import import_module

2023-10-27 11:08:52.287568: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-27 11:08:52.334542: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-27 11:08:52.335731: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-27 11:08:53.155501: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Might want to drop notebook for sampling because of jupyter and kernel overhead

In [2]:
file_name = "SamplesTest.csv"

# ONLY RUNS ON NEW SAMPLE GROUP
if not os.path.exists("dataframes/" + file_name):
    Table.create_empty_df(file_name, ["TimeStamp", "Name", "CPU_CORES", "CPU_CORES_LOG", "CPU_FREQ", "RAM_TOTAL_GB", "RAM_AV_GB", "RAM_USED_GB", "SWAP_TOTAL_GB", "SWAP_USED_GB", "CPU%", "RAM%", "RC", "WC", "RB", "WB", "dt", "PRED"])
else:
    # raise(FileExistsError)
    print("Sample file already created. Did you want to start a new batch?")

Sample file already created. Did you want to start a new batch?


In [3]:
# Create a TensorFlow session
config = tf.compat.v1.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
config.graph_options.rewrite_options.dependency_optimization = -1

# Set reduce_retracing to True or False
config.graph_options.rewrite_options.auto_mixed_precision = True

# Create a session with the specified configuration
sess = tf.compat.v1.Session(config=config)

Device mapping: no known devices.


# **ResNet50**

In [4]:
def Snapshot_ResNet50():
    from models.SS_ResNet50 import SS_ResNet50

    ss_resnet50 = SS_ResNet50()

    monitor = SystemMonitor()

    monitor.start(interval=0.1)
    ss_resnet50.predict()
    monitor.stop()

    Table.append_data_to_samples(file_name, "ResNet50", monitor, ss_resnet50)

Snapshot_ResNet50()


Monitoring started with an interval of 0.1 seconds.
1/1 [==============================] - 1s 1s/step
Monitoring stopped.
1 row(s) appended to SamplesTest.csv.
Appended to sample file:
['23:10:27:15:08:57', 'ResNet50', 4, 4, 2808.004, '11.68', '7.58', '3.68', '0.95', '0.00', 58.96153846153846, 34.96923076923076, 0.0, 1.9230769230769231, 0.0, 16384.0, 1.3, 'True']


# All Models at once

In [5]:
models_folder = "models"

# Get a list of all Python files in the folder (excluding __init__.py and SS_model.py)
model_files = [f[:-3] for f in os.listdir(models_folder) if f.endswith(".py") and f != "__init__.py" and f != "SS_model.py"]

# Ease of use print nr of found models
print("Found " + str(len(model_files)) + " models.")
for file in model_files:
    print(file)

# Count them for debug
nr_models_executed = 0

# Iterate through the model files and import the classes
for model_file in model_files:
    try:
        # Import the module dynamically
        model_module = import_module(f"{models_folder}.{model_file}")
        
        # Get the class name
        class_name = model_file.replace("Ss", "SS")
            
        # Create an instance of the model class
        model_instance = None
        _ctr = 0
        while True:
            # Some models give random errors so they must be run multiple times....
            _no_rand_error = True 

            # Handle model instantiation errors
            try:
                model_instance = getattr(model_module, class_name)()
            except AttributeError as e: # handle random error
                _no_rand_error = False
                _ctr += 1
                print("Execution failed on " + class_name + "... try " + str(_ctr))
            except Exception as e: # handle actual errors
                raise e

            if _no_rand_error:
                break
        
        if model_instance == None:
            raise ReferenceError

        # Instantiate a new system resource monitor
        monitor = SystemMonitor()
        
        # Start monitoring
        monitor.start(interval=0.1)
        # Make predictions
        model_instance.predict()
        # Stop monitoring
        monitor.stop()
        
        # Append data to the samples file for each model
        Table.append_data_to_samples(file_name, model_instance.__class__.__name__, monitor, model_instance)

        # Increment counter
        nr_models_executed += 1
    except Exception as e:
        print("Executed " + str(nr_models_executed) + " out of " + str(len(model_files)) + " models.")
        raise(e)

Found 37 models.
SS_EfficientNetV2B2
SS_ConvNeXtBase
SS_EfficientNetV2B0
SS_ResNet101
SS_EfficientNetB1
SS_ConvNeXtXLarge
SS_VGG19
SS_EfficientNetB5
SS_EfficientNetB2
SS_MobileNet
SS_ResNet50V2
SS_EfficientNetB4
SS_ConvNeXtTiny
SS_InceptionResNetV2
SS_VGG16
SS_EfficientNetB0
SS_EfficientNetV2L
SS_DenseNet201
SS_ConvNeXtSmall
SS_EfficientNetB7
SS_InceptionV3
SS_EfficientNetB6
SS_EfficientNetB3
SS_EfficientNetV2M
SS_DenseNet121
SS_ResNet152
SS_EfficientNetV2B3
SS_ResNet101V2
SS_MobileNetV2
SS_ResNet152V2
SS_DenseNet169
SS_NASNetLarge
SS_NASNetMobile
SS_ConvNeXtLarge
SS_ResNet50
SS_EfficientNetV2S
SS_Xception
Monitoring started with an interval of 0.1 seconds.
1/1 [==============================] - 2s 2s/step
Monitoring stopped.
1 row(s) appended to SamplesTest.csv.
Appended to sample file:
['23:10:27:15:09:04', 'SS_EfficientNetV2B2', 4, 4, 2808.004, '11.68', '7.62', '3.66', '0.95', '0.00', 46.769696969696966, 34.772727272727266, 0.0, 1.6666666666666667, 0.0, 12660.363636363636, 2.0000000

2023-10-27 11:09:13.456487: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f11803e03f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-10-27 11:09:13.456740: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-10-27 11:09:13.471533: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 [==============================] - 6s 6s/step
Monitoring stopped.
1 row(s) appended to SamplesTest.csv.
Appended to sample file:
['23:10:27:15:09:16', 'SS_ConvNeXtBase', 4, 4, 2808.004, '11.68', '7.28', '3.99', '0.95', '0.00', 49.23617021276595, 36.484042553191465, 0.1276595744680851, 1.9148936170212767, 1307.2340425531916, 31983.659574468085, 6.099999999999994, 'True']
Monitoring started with an interval of 0.1 seconds.
1/1 [==============================] - 2s 2s/step
Monitoring stopped.
1 row(s) appended to SamplesTest.csv.
Appended to sample file:
['23:10:27:15:09:21', 'SS_EfficientNetV2B0', 4, 4, 2808.004, '11.68', '7.74', '3.54', '0.95', '0.00', 49.770175438596475, 36.03596491228068, 0.2807017543859649, 10.324561403508772, 2443.2280701754385, 99956.77192982456, 2.0000000000000004, 'True']
Monitoring started with an interval of 0.1 seconds.
1/1 [==============================] - 2s 2s/step
Monitoring stopped.
1 row(s) appended to SamplesTest.csv.
Appended to sample file:
['23:

2023-10-27 11:10:23.649379: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-10-27 11:10:23.866029: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-10-27 11:10:23.976293: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-10-27 11:10:30.646462: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.


Monitoring started with an interval of 0.1 seconds.
1/1 [==============================] - 0s 391ms/step
Monitoring stopped.
1 row(s) appended to SamplesTest.csv.
Appended to sample file:
['23:10:27:15:10:31', 'SS_VGG19', 4, 4, 2808.004, '11.68', '5.51', '5.77', '0.95', '0.49', 46.51659919028341, 41.61862348178145, 0.242914979757085, 9.80161943319838, 1890.4615384615386, 88188.37246963562, 0.4, 'True']
Monitoring started with an interval of 0.1 seconds.
1/1 [==============================] - 4s 4s/step
Monitoring stopped.
1 row(s) appended to SamplesTest.csv.
Appended to sample file:
['23:10:27:15:10:40', 'SS_EfficientNetB5', 4, 4, 2808.004, '11.68', '7.77', '3.51', '0.95', '0.49', 45.619642857142864, 40.76178571428575, 0.22857142857142856, 8.689285714285715, 3423.0857142857144, 78087.31428571428, 3.3000000000000016, 'True']
Monitoring started with an interval of 0.1 seconds.
1/1 [==============================] - 2s 2s/step
Monitoring stopped.
1 row(s) appended to SamplesTest.csv.
App

2023-10-27 11:11:17.171543: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.


Monitoring started with an interval of 0.1 seconds.
1/1 [==============================] - 0s 356ms/step
Monitoring stopped.
1 row(s) appended to SamplesTest.csv.
Appended to sample file:
['23:10:27:15:11:25', 'SS_VGG16', 4, 4, 2808.004, '11.68', '7.10', '4.19', '0.95', '0.49', 42.64861460957178, 38.66574307304783, 403.7632241813602, 985.9093198992443, 11295818.801007556, 8060535.939546599, 0.4, 'True']
21834768/21834768 [==============================] - 7s 0us/step
Monitoring started with an interval of 0.1 seconds.
1/1 [==============================] - 1s 1s/step
Monitoring stopped.
1 row(s) appended to SamplesTest.csv.
Appended to sample file:
['23:10:27:15:11:36', 'SS_EfficientNetB0', 4, 4, 2808.004, '11.68', '7.92', '3.37', '0.95', '0.49', 42.22634146341461, 38.469268292682905, 609.4365853658536, 1271.6780487804879, 15094679.102439024, 10394589.03414634, 1.3, 'True']
478307632/478307632 [==============================] - 46s 0us/step
Monitoring started with an interval of 0.1 se

ValueError: in user code:

    File "/home/ubuntu/.local/lib/python3.8/site-packages/keras/src/engine/training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "/home/ubuntu/.local/lib/python3.8/site-packages/keras/src/engine/training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ubuntu/.local/lib/python3.8/site-packages/keras/src/engine/training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "/home/ubuntu/.local/lib/python3.8/site-packages/keras/src/engine/training.py", line 2283, in predict_step
        return self(x, training=False)
    File "/home/ubuntu/.local/lib/python3.8/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/ubuntu/.local/lib/python3.8/site-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compat